Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Gemini Pro 에서 Vertex AI Search를 Grounding Service 로 사용하는 방법.

Feedback : shins777@gmail.com. 

* 이 Colab은 Gemini Pro 의 기본 동작을 설명하기 위한 예제입니다.  


### 라이브러리 설치

In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [ ]:
model_name="gemini-1.0-pro"
project="ai-hangsik"
location="asia-northeast3"

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Tool, grounding

from IPython.display import display, Markdown

#### 초기화 및 모델 생성

In [ ]:
vertexai.init(project=project, location=location)
model = GenerativeModel(model_name)

In [ ]:
PROMPT = (
    "특별법에서 민간 부문의 양자정보통신기술개발 지원에 대해서 상세하게 설명해주세요."
)

#### Foundation model 내의 데이터를 기반으로 생성된 정보

In [ ]:
response = model.generate_content(PROMPT)
display(Markdown(response.candidates[0].text))

In [ ]:
DATA_STORE_PROJECT_ID = "ai-hangsik"
DATA_STORE_REGION = "global"
DATA_STORE_ID = "hackathon-ds_1709838247983"

#### Vertex AI Search 내의 데이터를 검색해서 생성된 정보

In [ ]:
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))